In [1]:
## ALL IMPORTS FOR A NEW NOTEBOOK
__SEED = 947
__N_FOLDS = 3
__NROWS = None

import os, sys, random, math
import matplotlib.pyplot as plt
# %matplotlib inline
plt.style.use('ggplot')

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)
import matplotlib.pylab as plt
import seaborn as sns
import itertools as it
import scipy
import glob
import matplotlib
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch.optim import Optimizer
import torchvision.transforms.transforms as txf
import torch.optim.lr_scheduler as lr_scheduler
from collections import OrderedDict

from sklearn import metrics
from sklearn import preprocessing as pp
from sklearn import model_selection as ms
from sklearn import impute

import ml_utils as mutils
import time, datetime, pickle

import pandas_profiling as pp


font = {'size'   : 13}
matplotlib.rc('font', **font)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mutils.seed_everything(__SEED)

fold1 = ms.StratifiedKFold(n_splits=__N_FOLDS, shuffle=True, random_state=__SEED)
fold2 = ms.StratifiedKFold(n_splits=__N_FOLDS, shuffle=True, random_state=__SEED+3)
fold3 = ms.StratifiedKFold(n_splits=__N_FOLDS, shuffle=True, random_state=__SEED+5)

sys.stderr = open('/dev/stdout', 'w')
sys.stderr = sys.__stderr__

In [2]:
from catboost import CatBoostClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

In [9]:
train = pd.read_csv("train.csv", index_col="id")
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [10]:
test = pd.read_csv("test.csv", index_col="id")
test.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,,,
600000,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,Piano,52f6dd16c,147d704e4,8d857a0a1,ca9ad1d4b,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0
600001,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,Bassoon,691ebeae8,8653dcc2e,67a8d4ebb,060a21580,7ca8775da,1.0,Novice,Cold,n,N,NaN,2.0,8.0
600002,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,Theremin,81f792c16,6cdda499e,69403e18c,165e81a00,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0
600003,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,Bassoon,c9134205b,acbca4827,cb681246b,77d41330d,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0
600004,0.0,0.0,1.0,F,Y,Red,Circle,NaN,Finland,Theremin,f0f100f57,6f800b9af,cd9feb5c6,2218d9dfe,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0


In [5]:
sample = train.sample(frac=0.1)
pp.ProfileReport(sample)

In [11]:
import category_encoders as ce

In [12]:
bin_features=[i for i in train.columns if i.split('_')[0]=='bin']
ord_features=[i for i in train.columns if i.split('_')[0]=='ord']
nom_features=[i for i in train.columns if i.split('_')[0]=='nom']

cyc_features=[i for i in train.columns if i in ['day','month']]

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin

class CrossValTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, transformer, cv):
        self.transformer = transformer
        self.cv = cv
    def fit(self, X, y=None):
        self.transformer.fit(X, y)
        return self
    def transform(self, X, y=None):
        df = X.copy()
        df = self.transformer.transform(X, y)
        return df
    def get_oof_transform(self, X, y):
        oof = pd.DataFrame(index=X.index, columns=X.columns)
        for train_idx, valid_idx in self.cv.split(X,y):
            train_X = X.loc[train_idx]
            train_y = y.loc[train_idx]
            valid_X = X.loc[valid_idx]
            valid_y = y.loc[valid_idx]
            
            self.transformer.fit(train_X, train_y)
            oof_txf = self.transformer.transform(valid_X)
            
            oof.loc[valid_idx] = oof_txf
        return oof

In [14]:
all_features = bin_features+ord_features+nom_features+cyc_features
TRG_PIPE = CrossValTransformer(ce.TargetEncoder(cols = all_features, smoothing=3.05), cv=ms.StratifiedKFold(n_splits=10))

In [15]:
train_enc = TRG_PIPE.get_oof_transform(train[all_features],train["target"])

In [16]:
TRG_PIPE.fit(train[all_features], train["target"])
test_enc = TRG_PIPE.transform(test[all_features])

In [17]:
def preprocessor(X, y, feat1, txf1, feat2, txf2, feat3, txf3, feat4, txf4, feat5, txf5, verbose=False):
    if(verbose):
        print("Preprocessing....")
    st = time.time()
#     preprocessed_x_1 = txf1.fit_transform(X[feat1], y).astype('float') # ordinal
    preprocessed_x_2 = txf2.fit_transform(X[feat2], y) # one hot
    preprocessed_x_3 = txf3.get_oof_transform(X[feat3], y).astype("float") # trg
    preprocessed_x_4 = txf4.get_oof_transform(X[feat4], y).astype("float") # cat
    preprocessed_x_5 = txf5.get_oof_transform(X[feat5], y).astype("float") # woe
    # merge preprocessed
    PPX = scipy.sparse.hstack([
#         preprocessed_x_1, 
        preprocessed_x_2,
        preprocessed_x_3, preprocessed_x_4,
        preprocessed_x_5]).tocsr()
    
    if verbose:
        print("Preprocessing....Done -- Time Taken: {}".format(datetime.timedelta(seconds=time.time()-st)))
    return PPX, y

In [21]:
def cross_val_train(model, X_train, y_train, cv, score_function, best_score, verbose=False):
    start_time = time.time()
    
    valid_scores = []
    train_scores = []
    for i, (train_idx, valid_idx) in enumerate(cv.split(X_train, y_train)):
        if verbose:
            print("Training....Fold {}".format(i+1))
        train_X = X_train.loc[train_idx]
        train_y = y_train.loc[train_idx]
        
        valid_X = X_train.loc[valid_idx]
        valid_y = y_train.loc[valid_idx]
        
        # fit and score
        model.fit(train_X, train_y)
        train_score = score_function(train_y, model.predict_proba(train_X)[:,1])
        valid_score = score_function(valid_y, model.predict_proba(valid_X)[:,1])
        
        if verbose:
            print("Fold {} train score: {:0.5f}".format(i+1, train_score))
            print("Fold {} valid score: {:0.5f}".format(i+1, valid_score))
        
        
        train_scores.append(train_score)
        valid_scores.append(valid_score)
    
    cv_score = np.array(valid_scores).mean()
    train_score = np.array(train_scores).mean()
    
    down = '\u2193'
    up = "\u2191"
    curarr = ""
    if (cv_score-best_score)>0:
        curarr = up
    elif (cv_score-best_score)<0:
        curarr = down

    print("{}CV valid score: {:.7f} - Previous best score: {:.7f} - Train score: {:6f} - Time {}".format(curarr, cv_score, best_score, train_score, str(datetime.timedelta(seconds=time.time()-start_time))))
    
    return cv_score

In [31]:
scoring = "roc_auc"

HistGBM_param = {
    'l2_regularization': 0.0,
    'loss': 'auto',
    'max_bins': 200,
    'max_depth': 15,
    'max_leaf_nodes': 31,
    'min_samples_leaf': 20,
    'n_iter_no_change': 50,
    'scoring': scoring,
    'tol': 1e-07,
    'validation_fraction': 0.15,
    'verbose': 0,
    'warm_start': False,
    "verbose":1
}

cbc = CatBoostClassifier(
    loss_function='CrossEntropy',
    eval_metric="AUC",
    task_type="CPU",
    learning_rate=0.05,
    n_estimators =100,   #5000
    early_stopping_rounds=10,
    random_seed=2019,
    verbose=1
)


estimators = [
        ('histgbm', HistGradientBoostingClassifier(**HistGBM_param)),
        ('catboost',cbc)
    ]



clf = StackingClassifier(
            estimators=estimators,
            final_estimator=LogisticRegression(verbose=1),
            )

In [ ]:
%%time
cross_val_train(HistGradientBoostingClassifier(**HistGBM_param), train_enc[all_features], train["target"], fold1, metrics.roc_auc_score, 0, verbose=True)

Training....Fold 1
Binning 0.063 GB of training data: 0.362 s
Binning 0.011 GB of validation data: 0.010 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 31 leaves, max depth = 6, train score: 0.69100, val score: 0.68829, in 0.038s
[2/100] 1 tree, 31 leaves, max depth = 7, train score: 0.69736, val score: 0.69694, in 0.035s
[3/100] 1 tree, 31 leaves, max depth = 7, train score: 0.71243, val score: 0.70651, in 0.038s
[4/100] 1 tree, 31 leaves, max depth = 7, train score: 0.71981, val score: 0.71301, in 0.040s
[5/100] 1 tree, 31 leaves, max depth = 7, train score: 0.72396, val score: 0.71694, in 0.037s
[6/100] 1 tree, 31 leaves, max depth = 8, train score: 0.72717, val score: 0.71991, in 0.042s
[7/100] 1 tree, 31 leaves, max depth = 7, train score: 0.73132, val score: 0.72407, in 0.043s
[8/100] 1 tree, 31 leaves, max depth = 9, train score: 0.73417, val score: 0.72634, in 0.043s
[9/100] 1 tree, 31 leaves, max depth = 7, train score: 0.73732, val score: 0.72998, in 0.039s
[10/100] 1 tre

[86/100] 1 tree, 31 leaves, max depth = 10, train score: 0.79222, val score: 0.77962, in 0.080s
[87/100] 1 tree, 31 leaves, max depth = 10, train score: 0.79237, val score: 0.77967, in 0.086s
[88/100] 1 tree, 31 leaves, max depth = 9, train score: 0.79254, val score: 0.77967, in 0.087s
[89/100] 1 tree, 31 leaves, max depth = 8, train score: 0.79258, val score: 0.77973, in 0.089s
[90/100] 1 tree, 31 leaves, max depth = 10, train score: 0.79274, val score: 0.77975, in 0.086s
[91/100] 1 tree, 31 leaves, max depth = 11, train score: 0.79280, val score: 0.77978, in 0.089s
[92/100] 1 tree, 31 leaves, max depth = 9, train score: 0.79293, val score: 0.77982, in 0.090s
[93/100] 1 tree, 31 leaves, max depth = 10, train score: 0.79307, val score: 0.77983, in 0.089s
[94/100] 1 tree, 31 leaves, max depth = 9, train score: 0.79316, val score: 0.77986, in 0.089s
[95/100] 1 tree, 31 leaves, max depth = 12, train score: 0.79326, val score: 0.77987, in 0.089s
[96/100] 1 tree, 31 leaves, max depth = 10, 

[68/100] 1 tree, 31 leaves, max depth = 10, train score: 0.80314, val score: 0.78383, in 0.072s
[69/100] 1 tree, 31 leaves, max depth = 10, train score: 0.80333, val score: 0.78396, in 0.082s
[70/100] 1 tree, 31 leaves, max depth = 11, train score: 0.80358, val score: 0.78414, in 0.075s
[71/100] 1 tree, 31 leaves, max depth = 10, train score: 0.80380, val score: 0.78419, in 0.075s
[72/100] 1 tree, 31 leaves, max depth = 10, train score: 0.80415, val score: 0.78443, in 0.071s
[73/100] 1 tree, 31 leaves, max depth = 11, train score: 0.80431, val score: 0.78451, in 0.078s
[74/100] 1 tree, 31 leaves, max depth = 9, train score: 0.80467, val score: 0.78463, in 0.082s
[75/100] 1 tree, 31 leaves, max depth = 9, train score: 0.80470, val score: 0.78474, in 0.083s
[76/100] 1 tree, 31 leaves, max depth = 9, train score: 0.80497, val score: 0.78486, in 0.079s
[77/100] 1 tree, 31 leaves, max depth = 9, train score: 0.80513, val score: 0.78503, in 0.085s
[78/100] 1 tree, 31 leaves, max depth = 10, 

In [ ]:
folds = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
fold_preds = np.zeros([test_encode.shape[0],3])
oof_preds = np.zeros([X.shape[0],3])
results = {}





# Fit Folds
f, ax = plt.subplots(1,3,figsize = [14,5])


for i, (trn_idx, val_idx) in enumerate(folds.split(X,y)):
    print(f"Fold {i} stacking....")

    clf = StackingClassifier(
            estimators=estimators,
            final_estimator=LogisticRegression(),
            )
    clf.fit(X.loc[trn_idx,:], y.loc[trn_idx])
    
    tmp_pred = clf.predict_proba(X.loc[val_idx,:])[:,1]
    
    oof_preds[val_idx,0] = tmp_pred
    
    fold_preds[:,0] += clf.predict_proba(test_encode)[:,1] / folds.n_splits
        
    estimator_performance = {}
    estimator_performance['stack_score'] = metrics.roc_auc_score(y.loc[val_idx], tmp_pred)
    
    for ii, est in enumerate(estimators):
            model = clf.named_estimators_[est[0]]
            pred = model.predict_proba(X.loc[val_idx,:])[:,1]
            oof_preds[val_idx, ii+1] = pred
            fold_preds[:,ii+1] += model.predict_proba(test_encode)[:,1] / folds.n_splits
            estimator_performance[est[0]+"_score"] = metrics.roc_auc_score(y.loc[val_idx], pred)
            
    stack_coefficients = {x+"_coefficient":y for (x,y) in zip([x[0] for x in estimators], clf.final_estimator_.coef_[0])}
    stack_coefficients['intercept'] = clf.final_estimator_.intercept_[0]
        
    results["Fold {}".format(str(i+1))] = [
            estimator_performance,
            stack_coefficients
        ]

    plot_roc_curve(clf, X.loc[val_idx,:], y.loc[val_idx], ax=ax[i])
    ax[i].plot([0.0, 1.0])
    ax[i].set_title("Fold {} - ROC AUC".format(str(i)))

    
    
    
    
    
    
    
    
plt.tight_layout(pad=2)
plt.show()

f, ax = plt.subplots(1,2,figsize = [11,5])
sns.heatmap(pd.DataFrame(oof_preds, columns = ['stack','histgbm','catboost']).corr(),
            annot=True, fmt=".2f",cbar_kws={'label': 'Correlation Coefficient'},cmap="magma",ax=ax[0])
ax[0].set_title("OOF PRED - Correlation Plot")
sns.heatmap(pd.DataFrame(fold_preds, columns = ['stack','histgbm','catboost']).corr(),
            annot=True, fmt=".2f",cbar_kws={'label': 'Correlation Coefficient'},cmap="inferno",ax=ax[1])
ax[1].set_title("TEST PRED - Correlation Plot")
plt.tight_layout(pad=3)
plt.show()